# Tutorial

This is a tutorial for using SPAMS - the Single Particle Model Solver.

For more advanced features please see the full documentation.

**Please note:** In this notebook code to be run in the command line is preceded by an exclamation mark - this is not needed if running the program outside of this notebook.

## Dependencies

*If you have read the dependencies section of the readme on github you can skip this section!*

In order to run the code (and this notebook), you will need to ensure you have the following installed:

- intel/2017.4.196-GCC-6.4.0-2.28
- impi/2017.3.196 imkl/2017.3.196
- imkl/2017.3.196
- netCDF-Fortran/4.4.4
- GCC/10.2.0
- Python/3.8.6
- numpy
- netCDF4
- matplotlib


For an scrtp managed system the following procedure will ensure correct dependendcies are installed:


    module purge; module load intel/2017.4.196-GCC-6.4.0-2.28  impi/2017.3.196 imkl/2017.3.196 netCDF-Fortran/4.4.4 GCC/10.2.0 Python/3.8.6
    pip3 install numpy netCDF4 Matplotlib


## Installing the code (downloading from git)

To install SPAMS, navigate to the directory in your file system where you would like to download it, and use the following command:

    git clone https://github.com/HetSys/PX915_GroupB_22-23

## Running the solver

The first time you use the program after downloading it from github, you will need to compile it first. This can be done by navigating to the SPAMS directory and using the command

In [1]:
! make

ifort `nf-config --fflags` -c nc_output.f90 -lnetcdf `nf-config --flibs`
ifort -O3 -c read_inputs.f90
ifort -O3 -c finite_diff_solver.f90
ifort -O3 `nf-config --fflags` -DMKL read_inputs.o nc_output.o finite_diff_solver.o `nf-config --flibs` -Wl,--start-group /warwick/desktop/2018/software/MPI/intel/2017.4.196-GCC-6.4.0-2.28/impi/2017.3.196/imkl/2017.3.196/mkl/lib/intel64/libmkl_intel_lp64.a /warwick/desktop/2018/software/MPI/intel/2017.4.196-GCC-6.4.0-2.28/impi/2017.3.196/imkl/2017.3.196/mkl/lib/intel64/libmkl_intel_thread.a /warwick/desktop/2018/software/MPI/intel/2017.4.196-GCC-6.4.0-2.28/impi/2017.3.196/imkl/2017.3.196/mkl/lib/intel64/libmkl_core.a -Wl,--end-group -liomp5 -lpthread -lm -ldl -lnetcdf -o finite_diff_solver 


To run the program enter the following command:

In [3]:
! python3 user_input.py

User input successful, calling solver...
 Success writing output file, user_input_output.nc                              
     

Solver executed successfully, plotting output...


## A simple input file

In order to use solver parameters specific to your system, you will need to edit the user input file.

In the main directory locate the file called 'user_input.py' and open it with your preferred text editor.

Here, we will go through the main parts of this input file line by line.

### Input file name

The default name for the input file is 'user_input', this can be changed if desired. You do not need to add a file extension, and there is a characters limit of 50.

*Please note changing this filename will change the command required to run the solver.*

In [1]:
# Change the name of this of this file containing the user input parameters.
solver_input_filename = 'user_input'

### Input Parameters 

Below, is a list of all available input parameters, along with their default values.

- **tsteps**: The total number of timesteps for which the simulation is run. This must be an integer value greater than 0, (default = 100). 
*Do not change this if reading iapp from file.*

- **dt**: The size of the timestep size in seconds. This must be a float with a value greater than 0 (default = 0.1).

- **c0**: Sets the initial concentration in mol m<sup>-3</sup>, This must be a float with a value greater than 0 (default = 0.0).

- **D**: The diffusion constant in m<sup>2</sup> s<sup>-1</sup>. This must be a float (default = 4.0e-15).

- **R**: The radius of the sphere used in the SPM in m. This must be a float with a value greater than 0 (default = 5.86e-6).

- **a**: Particle surface area per unit volume (m<sup>-1</sup>). This must be a float with a value greater than or equal to 0.

- **L**: Electrode thickness in meters. This must be a float with a value greater than or equal to 0 (default = 75.6e-6).

- **iapp**: Applied current density in A m<sup>2</sup>. This must be a 1D array of floats of length tsteps. The default sets a constant current density of 0.73*10<sup>-3</sup>.

- **iapp_label**: Labels the type of applied current density, e.g. constant or from file string.

- **electrode_charge**: Label of electrode charge as positive or negative. (default = p for positive electrode, n for negative electrode).

### Setting parameters

For setting system parameters there are two main options:

1. You can use the default parameters for a positive or negative electrode by calling UI.set_defaults_pos() or UI.set_defaults_neg().

In [ ]:
# Import default values
tsteps, dt, c0, D, R, a, L, iapp, iapp_label, electrode_charge = UI.set_defaults_pos()

2. You can manually set each of the parameters. 

    The block of code below can be used in place of importing the default values.

In [ ]:
# Number of timesteps, integer, greater than 0. Do not change if read iapp from file.
tsteps = 100
# Timestep size (s), real, greater than 0
dt = 0.1
# Initial concentration (mol m**-3), real, positive
c0 = 0.0
# Diffusion coefficient (m**2 s**-1), real
D = 4.0e-15
# Width of block (m), real, greater than 0
R = 5.86e-6
# Particle surface area per unit volume (m**-1), real, greater than 0
a = 3.821839e5
# Electrode thickness (m), real, greater than 0
L = 85.2e-6

#### Applying current

For varying the applied current density (i<sub>app</sub>) there are the following additional options:

1. The i<sub>app</sub> values can be read in from a csv file called 'iapp_filename'. This is the default setting in user_input.py (a default csv file is provided).


In [ ]:
# Read in applied current density from csv file
iapp_filename = 'WLTP_m10.csv'
iapp, iapp_label, tsteps = UI.iapp_read_csv(iapp_filename)

2. You can set a constant value for current density, 'iapp_const', by replacing the default csv file code with the code below.

In [ ]:
# Constant current - set iapp_const to a constant float value.
iapp_const = 0.73*10**(-3)
iapp, iapp_label = UI.iapp_constant_setup(tsteps, iapp_const)

3. You can set a stepped current density, 'iapp_steps', from a 2D array of values and timesteps using the code below.

In [ ]:
# Step function - create 2D array of step values and integer timesteps starting at timestep 0
iapp_steps = [[0.73*10**(-3), 0], [-0.73*10**(-3), int(tsteps/4)], [0.73*10**(-3), int(tsteps/2)]]
iapp, iapp_label = UI.iapp_step_setup(tsteps, iapp_steps)

4. A final option is to manually set the i<sub>app</sub> values,
(i<sub>app</sub> must be an array of floats with a length of tsteps.)

In [ ]:
###### Manually set up applied current ######
### Applied current (A m**2), real array of length tsteps

## Visualising Results

Any instuctions on non default visualisation should go here

In [ ]:
# Example plot

## Checkpointing

Information on how to restart from a checkpoint should go here

## More advanced features

parallelisation?

## Documentation for developers

Full documentation of the code is available on doxygen via...